In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT t1.user_id,
    CASE 
     WHEN t1.lt_day >= 365 THEN '1'
     ELSE '0'
    END AS is_new,
t1.lt_day,
t1.age,
t1.gender_segment,
t1.os_name,
t1.cpe_type_name,
t2.country,
t2.city,
substr(t3.title, instr(t3.title,' ')+1) AS age_segment,
substr(t4.title, instr(t3.title,' ')+1) AS traffic_segment,
substr(t5.title, instr(t3.title,' ')+1) AS lifetime_segment,
t1.nps_score, 
  CASE
    WHEN t1.nps_score >= 9 THEN 'promoters'
    WHEN t1.nps_score>= 7 THEN 'passieves'
    ELSE 'detractors'
  END AS nps_group
FROM user AS t1
JOIN location AS t2 ON t1.location_id = t2.location_id
JOIN age_segment AS t3 ON t1.age_gr_id = t3.age_gr_id
JOIN traffic_segment AS t4 ON t1.tr_gr_id = t4.tr_gr_id
JOIN lifetime_segment AS t5 ON t1.lt_gr_id = t5.lt_gr_id;

"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,is_new,lt_day,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,1,2320,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,1,2344,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,1,467,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Ссылка на презентацию: https://public.tableau.com/views/projectNPS_16880377197020/NPSResearch?:language=en-US&:display_count=n&:origin=viz_share_link